<a href="https://colab.research.google.com/github/AlyonaSarapina/python-and-cyber/blob/main/Testing_PythonProject_lambda_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Testing the lambda function code locally as a part of the PythonProject

# Connecting to AWS Simple Storage System (S3)


### Installing boto3 for use in this worksheet

In [4]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 12.0 MB/s eta 0:00:00


### Saving my access key, and secret key in an environment variable

In [1]:
import os
from IPython.display import clear_output

def set_environment_variable_values():
  ACCESS_KEY = input("Please enter the AWS access key: ")
  SECRET_ACCESS_KEY = input("Please enter the AWS secret access key: ")
  BUCKET_NAME = input("Please enter the name of the bucket in S3: ")
  os.environ['ACCESS_KEY'] = ACCESS_KEY
  os.environ['SECRET_ACCESS_KEY'] = SECRET_ACCESS_KEY
  os.environ['BUCKET_NAME'] = BUCKET_NAME
  clear_output()
  return None

set_environment_variable_values()


### Creating a connection to the S3 bucket

In [5]:
import boto3

def get_S3_client():
	resource = boto3.client(
     "s3",
		aws_access_key_id = os.environ.get('ACCESS_KEY'),
		aws_secret_access_key = os.environ.get('SECRET_ACCESS_KEY')
	)
	return resource

s3_client = get_S3_client()
print(s3_client)

### LAMBDA FUNCTIONS


In [41]:
import pandas as pd
import io

def add_school_data_to_bucket(filename, filedata):
    try:
      file_object =  io.StringIO()
      pd.DataFrame(filedata).to_csv(file_object, index=False)
      s3_client.put_object(Bucket=os.environ.get('BUCKET_NAME'), Body=file_object.getvalue(), Key=filename)
      return "New data successfully added", filedata
    except Exception as e:
      return str(e), []

def show_schools_data_in_bucket(filename):
    try:
        file_object = s3_client.get_object(Bucket=os.environ.get('BUCKET_NAME'), Key=filename)

        data_file = io.BytesIO(file_object['Body'].read())
        data = pd.read_csv(data_file)
        data_list = data.values.tolist()
        return "The data has been found", data_list
    except Exception as e:
        return "There is no such file", []




In [28]:
import json

def lambda_handler(event, context):
    global message, return_data, statuscode
    filename = "schools_list.csv"
    if event["httpMethod"] == "POST":
        if "body" in event.keys():
            request = event["body"]
            if type(request) is not dict:
                request = json.loads(request)
            if request is not None and "data" in request.keys():
                data = request["data"]
                if len(data) == 0:
                    message, return_data = "Please enter a valid data", []
                    statuscode = 404
                else:
                    message, return_data = add_school_data_to_bucket(filename, data)
                    statuscode = 200
            else:
                message, return_data = "Error in the POST request occured", []
                statuscode = 404
    elif event["httpMethod"] == "GET":
       message, return_data = show_schools_data_in_bucket(filename)
       statuscode = 200
    else:
        message, return_data = "Error occured", []
        statuscode = 404
    return {'statusCode': statuscode,
            'headers': {'Content-Type': 'application/json',
                        'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
                        'Access-Control-Allow-Methods': 'POST',
                        'Access-Control-Allow-Origin': '*'},
            'body': json.dumps({"message": message, "data": return_data})
            }


Event for POST data

In [39]:
event= {
    "httpMethod": "POST",
    "body": {
    "data": [
        [
            "Bannockburn Primary School & Nursery",
            51.4869172,
            0.1015561
        ],
        [
            "St Margaret Clitherow Primary School",
            51.50103289999999,
            0.1132992
        ]
    ]
}
}


lambda_handler(event, None)

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
  'Access-Control-Allow-Methods': 'POST',
  'Access-Control-Allow-Origin': '*'},
 'body': '{"message": "New data successfully added", "data": [["Bannockburn Primary School & Nursery", 51.4869172, 0.1015561], ["St Margaret Clitherow Primary School", 51.50103289999999, 0.1132992]]}'}

Event for GET data

In [40]:
event= {
    "httpMethod": "GET",
    "body": {
    "data": [
        [
            "Bannockburn Primary School & Nursery",
            51.4869172,
            0.1015561
        ],
        [
            "St Margaret Clitherow Primary School",
            51.50103289999999,
            0.1132992
        ]
    ]
}
}


lambda_handler(event, None)

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
  'Access-Control-Allow-Methods': 'POST',
  'Access-Control-Allow-Origin': '*'},
 'body': '{"message": "The data has been found", "data": [["Bannockburn Primary School & Nursery", 51.4869172, 0.1015561], ["St Margaret Clitherow Primary School", 51.50103289999999, 0.1132992]]}'}